### 0- Étape préliminaire: Importation des librairies

In [2]:
#general librairies
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd

#sklearn librairies
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.linear_model import *
from sklearn.ensemble import * 
from sklearn.svm import *
import statsmodels.api as sm
import xgboost as xgb

#appel a nos fonctions
from fcts_R.general import * 
from fcts_R.dataset_division import *

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


0.b - Importation du jeu de données

In [13]:
data0 = pd.read_csv("datasets_c/data0.csv")
data1 = pd.read_csv("datasets_c/data1.csv")


#On sépare les co-variables de la variable à prédire
X0,y0 = treatment(data0)
X1,y1 = treatment(data1)

#Data0: 
X_tr0, X_te0, y_tr0, y_te0= train_test_split(X0,y0, test_size=0.33,random_state=2023)

#Data1:
X_tr1, X_te1, y_tr1, y_te1= train_test_split(X1,y1, test_size=0.33,random_state=2023)

In [30]:
y_te = pred = np.row_stack((y_te0.to_numpy(),y_te1.to_numpy()))

Normalisation des données

In [6]:
#Pour data0: 
stdsc = StandardScaler()
X_tr_0 = stdsc.fit_transform(X_tr0)
X_te_0 = stdsc.transform(X_te0)

#Pour data1: 
stdsc = StandardScaler()
X_tr_1 = stdsc.fit_transform(X_tr1)
X_te_1 = stdsc.transform(X_te1)

Selection des meilleurs modèles par catégorie: 

Pour data0: 

In [57]:
pred_SVM0 =  NuSVR(C=1.5, gamma='scale', kernel='rbf', nu=0.6).fit(X_tr_0, y_tr0).predict(X_te_0)

pred_xgboost0 = xgb.XGBRegressor(seed = 2023, objective="reg:squarederror", n_estimators=100).fit(X_tr_0, y_tr0).predict(X_te_0)

pred_etrees0 = ExtraTreesRegressor(random_state=50, max_samples=None, n_estimators=1000).fit(X_tr_0, y_tr0).predict(X_te_0)

pred_bagg0 = BaggingRegressor(n_estimators= 2250, random_state=10).fit(X_tr_0, y_tr0).predict(X_te_0)

/Users/laurafuentesvicente/opt/anaconda3/envs/datachallenge/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/30/cz4hwnjn49sbv5x54df4kt4m0000gn/T/ipykernel_3874/2270472714.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  pred_etrees0 = ExtraTreesRegressor(random_state=50, max_samples=None, n_estimators=1000).fit(X_tr_0, y_tr0).predict(X_te_0)
/Users/laurafuentesvicente/opt/anaconda3/envs/datachallenge/lib/python3.11/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


Pour data1: 

In [8]:
data1 = pd.read_csv('datasets_c/data1.csv')

data_1 = data1.copy()
stdsc = StandardScaler()
data_1 = pd.DataFrame(stdsc.fit_transform(data_1), columns=data_1.columns)
data_1 = data_1.rename(columns={"volatile acidity":"volatile_acidity", "free sulfur dioxide": "free_sulfur_dioxide", "fixed acidity": "fixed_acidity", "citric acid": "citric_acid", "residual sugar":"residual_sugar", "total sulfur dioxide": "total_sulfur_dioxide"})

X = data_1[data_1.columns[0:-1]]
y = data_1["target"]
X_tr, X_te, y_tr, y_te= train_test_split(X,y, test_size=0.33,random_state=2023)

X_tr["target"] = y_tr

# Application de la regression lineaire
model_formula1 = "target ~ alcohol + sulphates + density  + volatile_acidity+ fixed_acidity + citric_acid + residual_sugar "
# Define the model
model1 = sm.OLS.from_formula(model_formula1, data=X_tr)

# Fit the model
results1 = model1.fit()

# View the regression summary
print(results1.summary())

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     60.95
Date:                Mon, 30 Oct 2023   Prob (F-statistic):           2.65e-69
Time:                        16:43:45   Log-Likelihood:                -910.07
No. Observations:                 761   AIC:                             1836.
Df Residuals:                     753   BIC:                             1873.
Df Model:                           7                                         
Covariance Type:            nonrobust  

In [10]:
pred_reglin1 = results1.predict(X_te)

pred_SVM1 = NuSVR(kernel="rbf", gamma="scale", nu=0.5,C=1).fit(X_tr_1,y_tr1).predict(X_te_1)

preds_XGB1 = xgb.XGBRFRegressor(n_estimators=420).fit(X_tr_1, y_tr1).predict(X_te_1)

pred_etrees1 = ExtraTreesRegressor(n_estimators=2750,random_state=50, max_samples=None).fit(X_tr_1, y_tr1).predict(X_te_1)

pred_rf1 = RandomForestRegressor(random_state=50, n_estimators=155,min_samples_split=2).fit(X_tr_1, y_tr1).predict(X_te_1)

/Users/laurafuentesvicente/opt/anaconda3/envs/datachallenge/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/30/cz4hwnjn49sbv5x54df4kt4m0000gn/T/ipykernel_3874/315787894.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  pred_etrees1 = ExtraTreesRegressor(n_estimators=2750,random_state=50, max_samples=None).fit(X_tr_1, y_tr1).predict(X_te_1)
/var/folders/30/cz4hwnjn49sbv5x54df4kt4m0000gn/T/ipykernel_3874/315787894.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  pred_rf1 = RandomForestRegressor(random_state=50, n_estimators=155,min_samples_split=2).fit

### On va chercher ainsi quelle est la combinaison avec plus forte performance totale: 

In [68]:
# data0:SVM 
#predsvm = build_pred(X_te0, X_te1, pred_SVM0,pred_etrees1)
#print("r2_score", r2_score(y_te,predsvm[:,1])) # 0.4191068436101377

#data0: xgboost
#predxgb = build_pred(X_te0, X_te1, pred_xgboost0,pred_etrees1)
#print("r2_score", r2_score(y_te,predxgb[:,1])) #0.4327753140486902

#data0: bagging
#predbag = build_pred(X_te0, X_te1, pred_bagg0,pred_etrees1)
#print("r2_score", r2_score(y_te,predbag[:,1])) #0.4816217721361392

#data0: extratrees
predetrees = build_pred(X_te0, X_te1, pred_etrees0,pred_etrees1)
print("r2_score", r2_score(y_te,predetrees[:,1])) #0.5020026463683728

r2_score 0.5020026463683728


#### **Modèle associé à la plus forte note est donc: extratrees pour les deux**

### Entrainement du modèle sélectionné et prediction sur X_test

On entraine nos modèles sélectionnés sur l'ensemble des données puis on crée une soumission

On crée d'abord X_test0 (avec que des vins de type 0), et X_test1 (resp. vins de type 1) pour pouvoir créer nos prédictions

In [ ]:
#On telecharge X_test
X_test = pd.read_csv("wine_test.csv")

idxtest0, idxtest1 = winetype(X_test)
X_test0, X_test1 = formal_div(X_test, idxtest0, idxtest1)

In [ ]:
#Data0: 
mod0 = ExtraTreesRegressor(random_state=50, max_samples=None, n_estimators=1000)
mod0.fit(X0,y0)
pred0 = mod0.predict(X_test0)


#Data1: 
mod1 = ExtraTreesRegressor(random_state=50, max_samples=None, n_estimators=2750)
mod1.fit(X1,y1)
pred1 = mod1.predict(X_test1)

#Creation 
pred = build_pred(X_test0, X_test1, pred0,pred1)

/var/folders/30/cz4hwnjn49sbv5x54df4kt4m0000gn/T/ipykernel_12692/228695055.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  mod0.fit(X0,y0)
/var/folders/30/cz4hwnjn49sbv5x54df4kt4m0000gn/T/ipykernel_12692/228695055.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  mod1.fit(X1,y1)


In [ ]:
import csv

    # Nommez les colonnes
column_names = ['wine_ID', 'target']
    # Spécifiez le nom du fichier CSV de sortie
csv_filename = 'submissions_R/'+'ExtraTrees_01'+'_'+'0010'+'.csv'
    # Ouvrez le fichier CSV en mode écriture et écrivez les données
with open(csv_filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
        # Écrivez les noms des colonnes
    writer.writerow(column_names)
        # Écrivez les données
    writer.writerows(pred)

In [ ]:
https://www.kaggle.com/code/qusaybtoush1990/wine-quality 